<a href="https://colab.research.google.com/github/patelmeshwa99/Sentiment-Analysis/blob/master/on_sentiment140_CNN_%2B_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!mkdir -p data
!wget -nc https://nyc3.digitaloceanspaces.com/ml-files-distro/v1/sentiment-analysis-is-bad/data/training.1600000.processed.noemoticon.csv.zip -P data
!unzip -n -d data data/training.1600000.processed.noemoticon.csv.zip

--2020-06-20 12:11:46--  https://nyc3.digitaloceanspaces.com/ml-files-distro/v1/sentiment-analysis-is-bad/data/training.1600000.processed.noemoticon.csv.zip
Resolving nyc3.digitaloceanspaces.com (nyc3.digitaloceanspaces.com)... 162.243.189.2
Connecting to nyc3.digitaloceanspaces.com (nyc3.digitaloceanspaces.com)|162.243.189.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85088192 (81M) [application/zip]
Saving to: ‘data/training.1600000.processed.noemoticon.csv.zip’

training.1600000.pr 100%[===================>]  81.15M  29.6MB/s    in 2.7s    

2020-06-20 12:11:49 (29.6 MB/s) - ‘data/training.1600000.processed.noemoticon.csv.zip’ saved [85088192/85088192]

Archive:  data/training.1600000.processed.noemoticon.csv.zip
  inflating: data/training.1600000.processed.noemoticon.csv  


In [2]:
import pandas as pd

df = pd.read_csv("data/training.1600000.processed.noemoticon.csv",
                names=['polarity', 'id', 'date', 'query', 'user', 'text'],
                encoding='latin-1')
df.head()

,polarity,id,date,query,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [3]:
df.polarity = df.polarity.replace({0: 0, 4: 1})
df.polarity.value_counts()

1    800000
0    800000
Name: polarity, dtype: int64

In [4]:
df = df.drop(columns=['id', 'date', 'query', 'user'])
df.head()

,polarity,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [5]:
df = df.sample(n=35000)
df.to_csv("data/sentiment140-subset.csv", index=False)

In [6]:
data = df[['text','polarity']]

In [7]:
!pip install tweet-preprocessor

In [8]:
import re

data = data[data.polarity != "Neutral"]
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

/usr/local/lib/python3.6/dist-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


In [9]:
sum(data["polarity"]==1)

17521

In [10]:
print(data[ data['polarity'] == 1].size)
print(data[ data['polarity'] == 0].size)

for idx,row in data.iterrows():
    row[0] = row[0].replace('rt',' ')

35042
34958


In [11]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Conv1D, MaxPooling1D, Dropout,Bidirectional,Flatten
from tensorflow.keras import regularizers
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

max_fatures = 12000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(df['text'].values)
X = tokenizer.texts_to_sequences(df['text'].values)
X = pad_sequences(X)

Using TensorFlow backend.


In [12]:
X.shape

(35000, 118)

In [13]:
from keras.layers import Conv2D
from keras.regularizers import l2
from keras.layers import Dense
from keras.regularizers import l2

embed_dim = 32
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
# model.add(Dropout(0.5))
# model.add(LSTM(100,return_sequences=True))
# model.add(Dense(64,activation='relu'))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(2,2))
model.add(LSTM(100,return_sequences=True))
# model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
# model.add(MaxPooling1D(2,2))
model.add(Flatten())
model.add(Dense(250,activation='relu'))
# model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
# model.add(MaxPooling1D(2,2))
# model.add(Dense(128,activation='relu'))
# model.add(Conv1D(filters=32, kernel_size=5, padding='same', activation='relu'))
# model.add(MaxPooling1D(2,2))
# model.add(Dense(16,activation='relu'))
# model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
# model.add(MaxPooling1D(2,2))
# model.add(LSTM(100))
model.add(Dropout(0.5))
# model.add(Flatten())
model.add(Dense(1,activation='sigmoid'))

# model.add(Conv1D(64,
#                  5,
#                  padding='valid',
#                  activation='relu',
#                  strides=1))


# model.add(Dense(1,activation='sigmoid', kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),
#     bias_regularizer=regularizers.l2(1e-4)))
# model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2, return_sequences=True))
# model.add(Dense(1,activation='sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 118, 32)           384000    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 118, 32)           3104      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 59, 32)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 59, 100)           53200     
_________________________________________________________________
flatten_1 (Flatten)          (None, 5900)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 250)               1475250   
_________________________________________________________________
dropout_1 (Dropout)          (None, 250)              

In [14]:
X_train, X_test, Y_train, Y_test = train_test_split(X,data.polarity, test_size = 0.2, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(28000, 118) (28000,)
(7000, 118) (7000,)


In [15]:
# validation_size = 3000

# X_validate = X_test[-validation_size:]
# Y_validate = Y_test[-validation_size:]
# X_test = X_test[:-validation_size]
# Y_test = Y_test[:-validation_size]


In [16]:
batch_size = 64
model.fit(X_train, Y_train, epochs = 3, batch_size=batch_size, verbose = 2, validation_data=(X_test, Y_test))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 28000 samples, validate on 7000 samples
Epoch 1/3
 - 54s - loss: 0.5367 - accuracy: 0.7248 - val_loss: 0.4833 - val_accuracy: 0.7649
Epoch 2/3
 - 46s - loss: 0.3928 - accuracy: 0.8278 - val_loss: 0.4902 - val_accuracy: 0.7650
Epoch 3/3
 - 46s - loss: 0.2895 - accuracy: 0.8805 - val_loss: 0.5752 - val_accuracy: 0.7567


In [17]:
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = 64)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

score: 0.58
acc: 0.76
